In [1]:
import pandas as pd
import numpy as np
import nltk
import time
import matplotlib.pyplot as plt
import seaborn as sns
import os, glob
import csv
from functools import reduce
import re
import string


In [4]:

labels = ["Direct Harassment", "Hate Speech","Sexual Harassment","Trolling", "Others", "Toxic"]

batch_files = ['usanews.csv', 'foxnews0.csv', 'foxnews1.csv', 'foxnews2.csv', 'foxnews3.csv', 
                'nogla0.csv','pew0.csv', 'rae0_0.csv', 
                 'rae0_1.csv', 'terror0_0.csv', 'terror0_1.csv']

vid_files = ["foxnews_FULL.csv", "rae_FULL.csv",  "terror_FULL.csv",
             "usanews.csv" , "pew0.csv", "nogla0.csv", "drdisrespect_FULL.csv"]
genre_files = ["gaming_channels_GENRE.csv", "news_channel_GENRE.csv" ]

files = []
files.extend(vid_files)
files.extend(genre_files)
print(files)

['foxnews_FULL.csv', 'rae_FULL.csv', 'terror_FULL.csv', 'usanews.csv', 'pew0.csv', 'nogla0.csv', 'drdisrespect_FULL.csv', 'gaming_channels_GENRE.csv', 'news_channel_GENRE.csv']


# TOP BI-GRAMS BASED ON TFIDF SCORES

In [35]:
src_folder = "tfidf_scores_WORDS_per_video/"
dest_folder = "tfidf_scores_WORDS_per_video/"
top_dfs = []
for file in files:
    vals = {}
    df = pd.read_csv(src_folder+file, index_col = 0)
    features = df.columns.difference(labels)
    dfs = []
    for label in labels:
        dl = df[df[label] == 1][features]
        dl = dl.T.sort_values(by=[dl.T.columns[0]], ascending = False).head(10)
        dl.index.name = "".join([w[0] for w in label.split()])
        if label == "Trolling":
            dl.index.name += "R"
        elif label == "Toxic":
            dl.index.name += "X"
        elif label == "Others":
            dl.index.name += "T"
        dl = dl.rename({dl.columns[0]:  dl.index.name + " value"}, axis='columns')
        dl.index.name += " words"
        dfs.append(dl)
    dfs = [d.reset_index(drop=False) for d in dfs]
    x = pd.concat(dfs, axis=1)
    display(x.style.set_caption(file))   
    x.to_csv(dest_folder+"TOP_WORDS_"+file)

,DH words,DH value,HS words,HS value,SH words,SH value,TR words,TR value,OT words,OT value,TX words,TX value
0,meltdown progress,2.309401,conservative tear,1.000000,beautiful kamala harris,0.447214,biden cheat,2.000000,traitor hang,1.506965,traitor hang,1.077060
1,magat meltdown,2.309401,fox total,1.000000,ivanka beautiful kamala,0.447214,gobble gobble,1.257449,civil war,0.860926,buck fiden,1.000000
2,magat meltdown progress,2.309401,sign traitor,1.000000,ivanka beautiful,0.447214,covid hoax,1.000000,need civil,0.601715,lose sucker,1.000000
3,maga tear,1.538530,jail right,1.000000,beautiful kamala,0.447214,hello dictatorship,1.000000,need civil war,0.601715,wait prison,1.000000
4,gobble gobble,1.517393,stimulus socialist,1.000000,kamala harris,0.447214,cheater bust,1.000000,hang gallow,0.595813,fk blm,1.000000
5,trump lol,1.474625,left garbage,1.000000,trump new,0.377964,raid server,1.000000,traitor hang treason,0.595813,fk acosta,1.000000
6,pardon turkey,1.425936,china virus,0.632233,big loser,0.377964,sign traitor,1.000000,hang treason,0.595813,pay mfer,1.000000
7,supreme court,1.302278,jail entire,0.577350,host big,0.377964,pay mfer,1.000000,traitor hang gallow,0.595813,swamp worm,1.000000
8,trump pardon,1.253666,entire left,0.577350,host big loser,0.377964,philadelphia rigger,1.000000,court traitor,0.475634,satan trump,1.000000
9,trump pardon turkey,1.084736,devil incarnate,0.577350,trump new host,0.377964,biden illuminati,1.000000,court traitor hang,0.475634,satan kuwait,1.000000


,DH words,DH value,HS words,HS value,SH words,SH value,TR words,TR value,OT words,OT value,TX words,TX value
0,bruh dude break,0.577350,00 hour,0.000000,toast thicc,0.447214,lesson old man,0.377964,00 hour,0.000000,00 hour,0.000000
1,bruh dude,0.577350,rae _raehype _raehype,0.000000,finally toast thicc,0.447214,teach lesson old,0.377964,rae _raehype _raehype,0.000000,rae _raehype _raehype,0.000000
2,dude break,0.577350,rae 2254,0.000000,toast thicc booty,0.447214,lesson old,0.377964,rae 2254,0.000000,rae 2254,0.000000
3,00 hour,0.000000,rae 2254 hug,0.000000,thicc booty,0.447214,time teach lesson,0.377964,rae 2254 hug,0.000000,rae 2254 hug,0.000000
4,rae _raehype,0.000000,rae 2nd,0.000000,finally toast,0.447214,time teach,0.377964,rae 2nd,0.000000,rae 2nd,0.000000
5,rae 2254 hug,0.000000,rae 2nd star,0.000000,rae 2nd,0.000000,teach lesson,0.377964,rae 2nd star,0.000000,rae 2nd star,0.000000
6,rae 2nd,0.000000,rae _rae100,0.000000,rae 2nd star,0.000000,old man,0.377964,rae _rae100,0.000000,rae _rae100,0.000000
7,rae 2nd star,0.000000,rae _raehype,0.000000,rae _rae100,0.000000,unlike video,0.301511,rae _raehype,0.000000,rae _raehype,0.000000
8,rae _rae100,0.000000,rae _raelove,0.000000,rae _raehype,0.000000,unsuscribe unlike,0.301511,rae _raelove,0.000000,rae _raelove,0.000000
9,rae _raelove,0.000000,pure support,0.000000,rae _raehype _raehype,0.000000,unlike video rae,0.301511,pure support,0.000000,pure support,0.000000


,DH words,DH value,HS words,HS value,SH words,SH value,TR words,TR value,OT words,OT value,TX words,TX value
0,die lmao,1.000000,black guy,2.852805,brian fan,1.000000,terroris simp,1.000000,kill nogla,1.000000,130 noob,1.000000
1,cory annoying,1.000000,guy nogla,1.340387,brian thicc,0.577350,dislike room,1.000000,burn wizard,1.000000,snitch fuck ridiculous,0.707107
2,lightning care,1.000000,guy nogla 2020,1.340387,brian thicc doe,0.577350,dislike trigger,1.000000,kill play,0.606875,snitch fuck,0.707107
3,sooo dirty,1.000000,nogla 2020,1.340387,thicc doe,0.577350,hope soon,0.577350,kill play guy,0.606875,snipe dickhead,0.627773
4,130 noob,1.000000,aussie weird,1.000000,release pleb,0.000000,trump2020 hope soon,0.577350,instant kill,0.577350,stream snipe dickhead,0.627773
5,breathes hes,0.707107,black guy nogla,0.927768,really sus,0.000000,trump2020 hope,0.577350,knife instant,0.577350,blocker fucker,0.622533
6,breathes hes impostor,0.707107,white guy nogla,0.480829,really think noah,0.000000,shoulda parent,0.447214,knife instant kill,0.577350,ad blocker fucker,0.622533
7,stream snipe dickhead,0.627773,white guy,0.422031,really want,0.000000,brown celebrity play,0.447214,play guy,0.513231,kill play,0.606875
8,snipe dickhead,0.627773,recently love terrorisor,0.000000,really want member,0.000000,orphan deserve shoulda,0.447214,leave open die,0.382034,kill play guy,0.606875
9,bad game watch,0.617562,recommendation midnight,0.000000,reason brian,0.000000,orphan deserve,0.447214,open die,0.382034,dang terroris,0.586032


,DH words,DH value,HS words,HS value,SH words,SH value,TR words,TR value,OT words,OT value,TX words,TX value
0,biden suck,8.291875,china joe,12.082489,lady gag,3.000000,joe biden,9.277306,sunny yu,1.755135,biden suck,11.167500
1,joe biden,5.582116,vote trump,9.542417,lady gaga,1.918114,china joe,3.793307,beijing biden,1.574272,joe biden,7.951498
2,china joe,4.976935,biden suck,9.167500,commie bitch,1.000000,biden prison,3.663920,life matter,1.397841,lady gaga,5.049229
3,biden jail,4.881563,vote biden,7.403092,buck fiden,1.000000,biden china,3.000000,biden voter,1.111717,cnn suck,4.000000
4,sleepy joe,4.760010,biden prison,6.777448,make gag,0.899653,moron biden,2.852340,terror attack,1.076648,moron biden,3.469787
5,biden racist,4.519776,joe biden,5.888062,gaga make gag,0.899653,beijing biden,2.734768,trump hate,1.000000,biden supporter,3.447166
6,lady gaga,4.387135,china pay,5.086143,gag lady,0.899653,vote trump,2.712668,47 year,1.000000,biden harris,3.331422
7,biden prison,4.061779,china biden,4.716271,make gag lady,0.899653,biden corrupt,2.567092,chy na,1.000000,vote biden,3.328542
8,biden crime,3.905258,moron biden,4.704680,gaga make,0.873213,biden win,2.513682,trump china,1.000000,vote trump,2.848222
9,biden crime family,3.462861,biden 2020,4.682197,joe kameltoe,0.751828,fake news,2.454750,austria war,1.000000,china joe,2.808168


,DH words,DH value,HS words,HS value,SH words,SH value,TR words,TR value,OT words,OT value,TX words,TX value
0,nancy cringe,1.000000,000 people,0.000000,felix gay,1.000000,members money,5.366563,000 people,0.000000,fake af,1.000000
1,nancy corpse,1.000000,response come,0.000000,require facetune picture,0.000000,charity members money,5.366563,response come,0.000000,try guy stfu,0.707107
2,nancy trash,1.000000,resize screen apparently,0.000000,resize screen apparently,0.000000,donations charity,5.366563,resize screen apparently,0.000000,try guy,0.707107
3,nancy boring,1.000000,respect btw,0.000000,respect btw,0.000000,donations charity members,5.366563,respect btw,0.000000,suck felix pick,0.577350
4,walk look like,0.424510,respect btw let,0.000000,respect btw let,0.000000,charity members,5.366563,respect btw let,0.000000,felix pick,0.577350
5,drunken man,0.424510,respond late,0.000000,respond late,0.000000,discord linked happened,0.638229,respond late,0.000000,suck felix,0.577350
6,way jack walk,0.424510,respond late everytime,0.000000,respond late everytime,0.000000,discord linked,0.638229,respond late everytime,0.000000,fck phasmo,0.333333
7,walk look,0.424510,respond ouija,0.000000,respond ouija,0.000000,join discord,0.430496,respond ouija,0.000000,fck phasmo audio,0.333333
8,way jack,0.393526,respond ouija board,0.000000,respond ouija board,0.000000,restart definitely,0.000000,respond ouija board,0.000000,lot time,0.333333
9,look like,0.352567,response come ask,0.000000,response come,0.000000,respond late,0.000000,response come ask,0.000000,audio lot time,0.333333


,DH words,DH value,HS words,HS value,SH words,SH value,TR words,TR value,OT words,OT value,TX words,TX value
0,tom castillo,1.000000,blm joke,8.535057,sexy dick,1.000000,blm terrorist,1.237311,00 fucking,0.000000,pete fuckeddddd,1.000000
1,nogla racist,1.000000,death blm,3.503489,want dick,0.707107,covid fake,1.000000,speech hater,0.000000,shut bitch,1.000000
2,mom sus,1.000000,blm racist,3.000000,want dick jk,0.707107,purge 2021,1.000000,speech chat,0.000000,death blm,1.000000
3,speedy suck,1.000000,lives matter,1.705081,shaving ball hurt,0.585874,biden like,1.000000,speech chat fun,0.000000,code fckubtch,1.000000
4,mrtop5 gayyyyy,1.000000,george floyd,1.368968,shaving ball,0.585874,warming myth,0.722942,speech damit,0.000000,sexy dick,1.000000
5,richie 21 gay,0.707107,blm suck,1.322003,want suck,0.577350,voting joe know,0.707107,speech die,0.000000,chat retard,1.000000
6,richie 21,0.707107,blm terrorist,1.000000,want suck dick,0.577350,trump gae biden,0.707107,speech die shut,0.000000,white boy,0.828362
7,pete fuckin,0.707107,poc blm,1.000000,suck dick,0.577350,voting joe,0.707107,speech free,0.000000,pete fuckin idiot,0.707107
8,pete fuckin idiot,0.707107,hate jews,1.000000,ban nogla,0.559914,eat kid,0.707107,speech free spam,0.000000,pete fuckin,0.707107
9,deluxe fuck,0.600376,osers arginalized,1.000000,dick live,0.529994,eat kid peel,0.707107,speech hater block,0.000000,shut blm fucker,0.707107


,DH words,DH value,HS words,HS value,SH words,SH value,TR words,TR value,OT words,OT value,TX words,TX value
0,people say member,2.309401,ni er,1.000000,baby meet,0.447214,000 000,0.000000,000 000,0.000000,non members,1.000000
1,people say,2.309401,000 000,0.000000,baby meet bedroom,0.447214,nice chat 3k,0.000000,nice chat 3k,0.000000,ew public chat,0.577350
2,say member,2.309401,nice chat 3k,0.000000,meet bedroom,0.447214,nice car,0.000000,nice car,0.000000,public chat,0.577350
3,member chat,1.374228,nice car,0.000000,hey baby,0.447214,nice 100 floor,0.000000,nice 100 floor,0.000000,ew public,0.577350
4,anti member,1.188723,nice 100 floor,0.000000,hey baby meet,0.447214,nice 100,0.000000,nice 100,0.000000,_flappin _flappin,0.475917
5,anti member chat,1.188723,nice 100,0.000000,000 000,0.000000,ni er,0.000000,ni er,0.000000,start play trash,0.336108
6,non members,1.000000,news chance free,0.000000,new song drop,0.000000,news chance free,0.000000,news chance free,0.000000,live 22 minute,0.336108
7,doc boomer,1.000000,news chance,0.000000,nice 100 floor,0.000000,news chance,0.000000,news chance,0.000000,live 22,0.336108
8,want member,0.611136,new transition doc,0.000000,nice 100,0.000000,new transition doc,0.000000,new transition doc,0.000000,22 minute,0.336108
9,bro bros trash,0.600742,new transition,0.000000,ni er,0.000000,new transition,0.000000,new transition,0.000000,minute start,0.336108


,DH words,DH value,HS words,HS value,SH words,SH value,TR words,TR value,OT words,OT value,TX words,TX value
0,people say member,2.372892,blm joke,8.104467,ban nogla,1.000000,members money,5.366563,play guy,1.000000,white boy,1.566457
1,say member,2.351603,death blm,4.000000,live matter,1.000000,donations charity,5.366563,kill nogla,1.000000,stream snipe,1.000000
2,people say,2.199852,black guy,3.505535,felix gay,1.000000,charity members,5.366563,guy lol,1.000000,public chat,1.000000
3,member chat,1.264009,blm racist,3.000000,meet bedroom,1.000000,charity members money,5.366563,00 rae,0.000000,chat retard,1.000000
4,anti member chat,1.236869,lives matter,2.342428,00 rae,0.000000,donations charity members,5.366563,potato powers activate,0.000000,die lol,1.000000
5,anti member,1.236869,blm suck,2.000000,potato pick,0.000000,blm terrorist,1.483568,potato noise,0.000000,blm suck,1.000000
6,suck bad,1.000000,death black,1.489961,potato potato,0.000000,purge 2021,1.000000,potato pick,0.000000,non members,1.000000
7,bro bros,1.000000,george floyd,1.457013,potato potato family,0.000000,biden like,1.000000,potato potato,0.000000,ad blocker,1.000000
8,tom castillo,1.000000,guy nogla 2020,1.443434,potato potato potato,0.000000,ban nogla,1.000000,potato potato family,0.000000,death blm,1.000000
9,smell like,1.000000,guy nogla,1.411289,potato powers,0.000000,join discord,1.000000,potato potato potato,0.000000,guy stfu,1.000000


,DH words,DH value,HS words,HS value,SH words,SH value,TR words,TR value,OT words,OT value,TX words,TX value
0,biden suck,8.291081,china joe,12.101228,lady gag,3.000000,joe biden,10.618480,traitor hang,3.749889,biden suck,11.164323
1,joe biden,7.532736,vote trump,9.403320,lady gaga,1.919480,biden prison,3.807546,civil war,2.000000,joe biden,7.898715
2,sleepy joe,5.903194,biden suck,9.164323,make gag,1.007282,china joe,3.801986,sunny yu,1.754881,lady gaga,5.050634
3,china joe,4.986737,vote biden,7.472942,gag lady,1.007282,vote trump,3.723379,beijing biden,1.574307,cnn suck,4.000000
4,biden jail,4.882732,biden prison,6.890406,make gag lady,1.007282,biden win,3.014630,biden voter,1.465081,vote biden,3.875534
5,lady gaga,4.371027,joe biden,6.149795,commie bitch,1.000000,biden china,3.000000,life matter,1.398097,biden harris,3.518930
6,biden racist,4.315291,china pay,5.086512,kamala harris,1.000000,fake news,2.942018,terror attack,1.077138,moron biden,3.477117
7,biden prison,4.063162,china virus,4.946780,trump new,1.000000,moron biden,2.857837,mr president,1.000000,biden supporter,3.350534
8,biden crime,3.780495,biden 2020,4.941256,buck fiden,1.000000,biden harris,2.829684,say biden,1.000000,vote trump,2.965617
9,crime family,3.464373,china biden,4.716686,gaga make,0.977828,vote biden,2.787209,biden obama,1.000000,china joe,2.813161


# TOP BI-GRAMS BASED ON LOWEST P-VALUE

In [3]:
genre_files = ["gaming_channels_GENRE.csv", "news_channel_GENRE.csv" ]
src_folder = "corr_words_per_label/"
corr_dfs = []
for file in files:
    df = pd.read_csv(src_folder+file, index_col = 0) 
    df.dropna(axis = 0, how = 'all', inplace = True)
    
    dic = {}
    dfs = {}
    #corr coeffcorr coeff
    for label in labels:
        dp = df[(df[label + " p-val"] > 0 )& ( df[label + " corr coeff"] > 0)].sort_values(by=[label + " p-val"], ascending = True).head(5)
        top_features = list(dp[label + " p-val"].index)
        #display(df[label + " p-val"])
        #dt = dp[ [label + " p-val", label + " corr coeff"]].T[top_features]
        dic[label] = top_features
   
    
    corr_coeff = {}
    for label in labels:
        col = ["top " + label]
        col.extend(dic[label])
        corr_df = pd.DataFrame(columns= col )
        
        for row in labels:
            new_row = {}
            for word in dic[label]:
                x = df.loc[word, row + " corr coeff"]
                row_label = "".join([w[0] for w in row.split()])
                
                if row == "Trolling":
                    row_label += "R"
                elif row == "Toxic":
                    row_label += "X"
                elif row == "Others":
                    row_label += "T" 
                
                new_row["top " + label] = row_label
                new_row[word] = x
            corr_df = corr_df.append(new_row, ignore_index=True)
        
        display(corr_df.set_index("top " + label).T.style.set_caption(file))
    

top Direct Harassment,DH,HS,SH,TR,OT,TX
bye bye,0.774597,-0.309839,-0.309839,0.464758,-0.309839,-0.309839
fox news,0.774597,-0.309839,-0.309839,0.464758,-0.309839,-0.309839
biden win,0.774597,-0.309839,-0.309839,0.464758,-0.309839,-0.309839
wear mask,0.774597,-0.309839,-0.309839,-0.309839,0.464758,-0.309839
president elect,0.774597,0.464758,-0.309839,-0.309839,-0.309839,-0.309839


top Hate Speech,DH,HS,SH,TR,OT,TX
country communist,-0.309839,0.774597,-0.309839,0.464758,-0.309839,-0.309839
dead people,-0.309839,0.774597,-0.309839,0.464758,-0.309839,-0.309839
dead people vote,-0.309839,0.774597,-0.309839,0.464758,-0.309839,-0.309839
lose lawsuit katie,0.632456,0.632456,-0.316228,-0.316228,-0.316228,-0.316228
manchurian candidate,0.632456,0.632456,-0.316228,-0.316228,-0.316228,-0.316228


top Sexual Harassment,DH,HS,SH,TR,OT,TX
beautiful kamala,0.632456,-0.316228,0.632456,-0.316228,-0.316228,-0.316228
beautiful kamala harris,0.632456,-0.316228,0.632456,-0.316228,-0.316228,-0.316228
big loser,0.632456,-0.316228,0.632456,-0.316228,-0.316228,-0.316228
host big,0.632456,-0.316228,0.632456,-0.316228,-0.316228,-0.316228
host big loser,0.632456,-0.316228,0.632456,-0.316228,-0.316228,-0.316228


top Trolling,DH,HS,SH,TR,OT,TX
biden cheat,0.464758,-0.309839,-0.309839,0.774597,-0.309839,-0.309839
biden voter,-0.309839,0.464758,-0.309839,0.774597,-0.309839,-0.309839
talk gobbles,0.417365,-0.417365,-0.417365,0.695608,-0.417365,0.139122
pardon turkey pardon,0.417365,-0.417365,-0.417365,0.695608,-0.417365,0.139122
pardon trump pardon,0.417365,-0.417365,-0.417365,0.695608,-0.417365,0.139122


top Others,DH,HS,SH,TR,OT,TX
traitor hang,0.282843,-0.424264,-0.424264,-0.424264,0.707107,0.282843
civil war,-0.417365,0.139122,-0.417365,0.417365,0.695608,-0.417365
sock presses mouth,0.632456,-0.316228,-0.316228,-0.316228,0.632456,-0.316228
war begin,-0.316228,-0.316228,-0.316228,0.632456,0.632456,-0.316228
mr president,-0.316228,0.632456,-0.316228,-0.316228,0.632456,-0.316228


...

top Hate Speech,DH,HS,SH,TR,OT,TX
kick joe,-0.309839,0.774597,-0.309839,0.464758,-0.309839,-0.309839
safe america,-0.309839,0.774597,-0.309839,-0.309839,-0.309839,0.464758
ruin california vote,-0.309839,0.774597,-0.309839,0.464758,-0.309839,-0.309839
ruin california,-0.309839,0.774597,-0.309839,0.464758,-0.309839,-0.309839
indianapolis countyworkers citizen,-0.309839,0.774597,-0.309839,0.464758,-0.309839,-0.309839


top Sexual Harassment,DH,HS,SH,TR,OT,TX
make gag,-0.309839,-0.309839,0.774597,-0.309839,-0.309839,0.464758
gag lady,-0.309839,-0.309839,0.774597,-0.309839,-0.309839,0.464758
make gag lady,-0.309839,-0.309839,0.774597,-0.309839,-0.309839,0.464758
gaga make,0.417365,-0.417365,0.695608,-0.417365,-0.417365,0.139122
hell kameltoe,0.632456,-0.316228,0.632456,-0.316228,-0.316228,-0.316228


top Trolling,DH,HS,SH,TR,OT,TX
blm biden laptop,-0.309839,0.464758,-0.309839,0.774597,-0.309839,-0.309839
socialism communism,-0.309839,0.464758,-0.309839,0.774597,-0.309839,-0.309839
steal election,-0.309839,0.464758,-0.309839,0.774597,-0.309839,-0.309839
biden run,0.464758,-0.309839,-0.309839,0.774597,-0.309839,-0.309839
biden love,-0.309839,-0.309839,-0.309839,0.774597,-0.309839,0.464758


top Others,DH,HS,SH,TR,OT,TX
know talk,0.464758,-0.309839,-0.309839,-0.309839,0.774597,-0.309839
terror attack,-0.309839,-0.309839,-0.309839,0.464758,0.774597,-0.309839
dem want,-0.309839,-0.309839,-0.309839,0.464758,0.774597,-0.309839
attack vienna,-0.309839,-0.309839,-0.309839,0.464758,0.774597,-0.309839
attack austria,-0.424264,0.282843,-0.424264,0.282843,0.707107,-0.424264


top Toxic,DH,HS,SH,TR,OT,TX
žðÿ žðÿ žðÿ,-0.309839,0.464758,-0.309839,-0.309839,-0.309839,0.774597
bin biden crime,-0.309839,0.464758,-0.309839,-0.309839,-0.309839,0.774597
žðÿ žðÿ,-0.309839,0.464758,-0.309839,-0.309839,-0.309839,0.774597
kamala black,-0.309839,0.464758,-0.309839,-0.309839,-0.309839,0.774597
kamala black dementia,-0.309839,0.464758,-0.309839,-0.309839,-0.309839,0.774597


# TOP POS SEQUENCE

In [9]:
genre_files = ["gaming_channels_GENRE.csv", "news_channel_GENRE.csv" ]
src_folder =  "tfidf_scores_POS_per_video/"
dest_folder =  "0_FOR_ANALYSIS_RESULTS"
for file in files:
    vals = {}
    df = pd.read_csv(src_folder+file, index_col = 0)
    features = df.columns.difference(labels)
    dfs = []
    for label in labels: 
        dl = df[df[label] == 1][features]
        dl = dl.T.sort_values(by=[dl.T.columns[0]], ascending = False).head(10)
        dl.index.name = "".join([w[0] for w in label.split()])
        if label == "Trolling":
            dl.index.name += "R"
        elif label == "Toxic":
            dl.index.name += "X"
        elif label == "Others":
            dl.index.name += "T"
        dl = dl.rename({dl.columns[0]:  dl.index.name + " value"}, axis='columns')
        dl.index.name += " POS-SEQ"
        dfs.append(dl)
        
    dfs = [d.reset_index(drop=False) for d in dfs]
    x = pd.concat(dfs, axis=1)
    display(x.style.set_caption(file))   
    x.to_csv(dest_folder+"TOP_POS_TFIDF_"+file)

,DH POS-SEQ,DH value,HS POS-SEQ,HS value,SH POS-SEQ,SH value,TR POS-SEQ,TR value,OT POS-SEQ,OT value,TX POS-SEQ,TX value
0,noun noun,15.214435,noun noun,3.904535,propn adj propn propn,0.565349,verb noun,9.279082,noun verb,2.005836,propn propn,4.839207
1,verb noun,14.142918,noun verb,3.886879,propn adj propn,0.483853,propn propn,9.218137,noun verb noun,1.990159,noun noun,4.593376
2,propn propn,13.752360,adj noun,3.450634,verb adj noun adj,0.459523,noun verb,7.906531,verb noun,1.882741,verb noun,4.190548
3,noun verb,12.100911,verb noun,3.169096,adj propn propn,0.457393,propn verb,6.147047,verb adj,1.103849,noun verb,2.560507
4,adj noun,10.257413,verb adj,2.209510,adj noun adj noun,0.416604,adj noun,5.966855,adj noun,1.062969,adj noun,2.047748
5,propn verb,9.604407,noun verb noun,2.193742,adj noun,0.388600,propn propn propn,5.014738,verb adj noun,0.982086,noun verb noun,2.030822
6,verb propn,7.348932,verb propn,1.956459,adj noun adj,0.384343,propn noun,4.718844,verb noun propn,0.844086,propn emoji,1.803802
7,propn noun,7.247761,noun verb propn,1.892717,adj propn,0.320698,noun propn,4.608454,noun noun,0.774325,emoji propn,1.695973
8,verb verb,6.164361,verb adj noun,1.795612,noun adj noun,0.317437,noun noun,4.366951,noun propn,0.755990,propn emoji propn,1.523291
9,noun verb noun,5.858246,noun adj,1.771781,propn adj,0.309786,noun adj,4.019572,adj noun verb,0.709028,adj noun noun,1.445135


,DH POS-SEQ,DH value,HS POS-SEQ,HS value,SH POS-SEQ,SH value,TR POS-SEQ,TR value,OT POS-SEQ,OT value,TX POS-SEQ,TX value
0,noun noun verb,0.738717,adj adj,0.000000,adv verb noun noun,0.478684,noun verb noun adj,0.403660,adj adj,0.000000,adj adj,0.000000
1,noun verb,0.517897,propn adj adv adj,0.000000,noun noun,0.387894,verb noun adj noun,0.358982,propn adj adv adj,0.000000,propn adj adv adj,0.000000
2,noun noun,0.431369,propn adj num,0.000000,verb noun noun noun,0.386971,adj noun verb noun,0.358982,propn adj num,0.000000,propn adj num,0.000000
3,propn adj num,0.000000,propn adj noun verb,0.000000,adv verb noun,0.385265,adj noun,0.349483,propn adj noun verb,0.000000,propn adj noun verb,0.000000
4,propn adj noun propn,0.000000,propn adj noun propn,0.000000,adv verb,0.324196,verb noun adj,0.330838,propn adj noun propn,0.000000,propn adj noun propn,0.000000
5,propn adj noun pron,0.000000,propn adj noun pron,0.000000,noun noun noun,0.303295,adj noun verb,0.299483,propn adj noun pron,0.000000,propn adj noun pron,0.000000
6,propn adj noun num,0.000000,propn adj noun num,0.000000,verb noun noun,0.291009,propn adp noun propn,0.288617,propn adj noun num,0.000000,propn adj noun num,0.000000
7,propn adj noun noun,0.000000,propn adj noun noun,0.000000,verb noun,0.201112,noun propn verb pron,0.288617,propn adj noun noun,0.000000,propn adj noun noun,0.000000
8,propn adj noun adv,0.000000,propn adj noun adv,0.000000,propn adj adv,0.000000,propn verb pron adv,0.288617,propn adj noun adv,0.000000,propn adj noun adv,0.000000
9,propn adj noun,0.000000,propn adj noun,0.000000,propn adj noun pron,0.000000,adp noun propn verb,0.288617,propn adj noun,0.000000,propn adj noun,0.000000


,DH POS-SEQ,DH value,HS POS-SEQ,HS value,SH POS-SEQ,SH value,TR POS-SEQ,TR value,OT POS-SEQ,OT value,TX POS-SEQ,TX value
0,noun verb,2.840746,propn propn,2.077250,propn noun,1.000000,adj noun,1.161807,verb propn,2.000000,noun verb,2.885194
1,noun adj,2.073149,propn propn num,1.416033,noun noun,1.000000,verb noun,0.856816,verb noun,0.938076,verb noun,1.901770
2,noun verb noun,2.009364,propn num,1.158583,propn verb noun,0.719953,noun verb verb,0.739419,verb noun emoji,0.749230,noun verb noun,1.782150
3,propn propn,1.957917,propn propn propn num,1.119729,propn verb,0.557049,pron verb noun,0.704693,verb noun verb,0.586866,adj noun verb,1.370301
4,adj noun,1.691532,adj noun,0.755121,verb noun,0.413962,verb verb,0.670353,noun emoji,0.582822,adj noun,1.280482
5,verb noun,1.665774,propn propn propn,0.701205,pron verb propn propn,0.000000,pron verb,0.652752,pron verb verb noun,0.564421,num noun,1.000000
6,propn noun,1.226438,num adj noun,0.696275,pron verb propn adv,0.000000,pron noun pron,0.613243,pron verb verb,0.468222,propn noun,1.000000
7,adj verb,1.169106,noun adj noun,0.693832,pron verb propn emoji,0.000000,noun pron,0.598585,verb verb noun,0.435543,num propn,1.000000
8,adj adj,1.000000,num adj,0.646823,pron verb propn noun,0.000000,noun verb verb noun,0.575678,noun verb,0.417064,noun noun,0.984694
9,num propn,1.000000,noun propn propn num,0.611911,pron verb propn num,0.000000,noun verb,0.476570,noun verb noun verb,0.371931,verb adj,0.935923


,DH POS-SEQ,DH value,HS POS-SEQ,HS value,SH POS-SEQ,SH value,TR POS-SEQ,TR value,OT POS-SEQ,OT value,TX POS-SEQ,TX value
0,propn propn,65.250636,propn propn,107.415819,propn propn,3.849562,propn propn,52.072118,propn propn,9.076297,propn propn,49.716911
1,propn noun,36.329188,noun noun,66.168020,propn verb,2.734686,noun noun,33.039831,propn propn propn,6.130309,propn noun,35.149994
2,noun noun,33.447047,propn noun,63.759039,verb noun,2.612700,propn noun,31.527701,noun verb,5.574704,noun noun,33.850698
3,propn propn propn,27.779089,propn propn propn,55.000539,propn propn propn,2.366538,propn verb,28.053288,propn verb,4.157561,noun verb,28.408533
4,propn verb,27.203888,noun propn,50.310178,noun propn,2.000523,noun verb,25.095223,propn noun,4.064130,propn verb,27.582268
5,noun propn,21.060100,propn verb,46.083079,propn noun,1.969265,verb noun,24.254338,noun propn,3.821431,propn propn propn,24.119156
6,noun verb,19.415276,noun verb,43.251854,noun noun,1.556217,propn propn propn,24.210189,verb noun,3.646273,noun propn,21.982580
7,verb propn,16.292991,verb noun,36.421450,propn verb pron,1.384157,noun propn,22.265399,verb propn,3.472509,verb noun,19.430947
8,verb noun,15.758408,verb propn,32.933535,verb pron,1.131626,verb propn,21.519951,verb verb,2.752992,verb propn,19.221245
9,propn adj,12.722998,adj noun,30.551107,propn propn propn propn,1.042961,propn verb noun,14.633950,propn propn propn propn,2.743392,adj noun,16.279922


,DH POS-SEQ,DH value,HS POS-SEQ,HS value,SH POS-SEQ,SH value,TR POS-SEQ,TR value,OT POS-SEQ,OT value,TX POS-SEQ,TX value
0,noun noun,3.000000,adj adj,0.000000,propn adj,1.000000,propn verb noun propn,4.498817,adj adj,0.000000,adj adp,1.000000
1,noun verb,2.000000,propn adv noun adj,0.000000,adj adj,0.000000,verb noun propn noun,4.461999,propn adv noun adj,0.000000,noun verb,0.878345
2,verb verb adp adj,0.353904,propn adv intj,0.000000,propn adv adj noun,0.000000,propn propn verb noun,4.346861,propn adv intj,0.000000,verb noun verb,0.715940
3,noun propn verb verb,0.338794,propn adv emoji emoji,0.000000,propn adv noun,0.000000,noun propn noun,4.065723,propn adv emoji emoji,0.000000,noun verb propn,0.542390
4,propn verb verb adp,0.338794,propn adv emoji,0.000000,propn adv intj,0.000000,verb noun propn,3.890148,propn adv emoji,0.000000,verb noun,0.453162
5,verb adp adj noun,0.302243,propn adv adv emoji,0.000000,propn adv emoji emoji,0.000000,propn propn verb,3.719869,propn adv adv emoji,0.000000,verb propn verb pron,0.420909
6,verb adp adj,0.297854,propn adv adv adv,0.000000,propn adv emoji,0.000000,propn verb noun,3.212620,propn adv adv adv,0.000000,propn noun noun noun,0.394314
7,verb verb adp,0.297854,propn adv adv adj,0.000000,propn adv adv emoji,0.000000,noun propn,2.835635,propn adv adv adj,0.000000,verb propn,0.390764
8,adp adj noun,0.281389,propn adv adv,0.000000,propn adv adv adv,0.000000,propn verb,2.529490,propn adv adv,0.000000,verb propn noun noun,0.389090
9,adp adj,0.272023,propn adv adp propn,0.000000,propn adv adv adj,0.000000,propn noun,2.376905,propn adv adp propn,0.000000,propn verb pron adv,0.372217


,DH POS-SEQ,DH value,HS POS-SEQ,HS value,SH POS-SEQ,SH value,TR POS-SEQ,TR value,OT POS-SEQ,OT value,TX POS-SEQ,TX value
0,propn verb,2.478702,propn verb,34.109429,verb verb,1.049164,verb verb,2.108437,adj adj,0.000000,propn noun,2.702589
1,noun noun,2.356125,propn noun,12.967063,adj propn,1.000000,verb noun,2.090796,pron verb intj num,0.000000,noun noun,2.666345
2,propn noun,1.504566,noun noun,10.139659,verb verb noun,0.704944,noun verb,1.603098,pron verb adv pron,0.000000,noun verb,2.636386
3,propn propn,1.351146,propn propn,5.197985,noun verb verb,0.646155,propn adj,1.253062,pron verb adv propn,0.000000,verb noun,1.607606
4,verb adj,1.255781,noun verb,3.827149,adj noun verb verb,0.614568,propn noun,1.157240,pron verb adv verb,0.000000,propn propn,1.464876
5,adj noun,1.174659,propn adj,3.382538,pron verb propn propn,0.580473,noun verb noun,1.145865,pron verb emoji,0.000000,adj propn,1.227256
6,propn verb adj,1.084877,adv verb,2.441015,verb noun,0.546695,noun adj,1.128792,pron verb emoji emoji,0.000000,noun propn,1.189371
7,propn propn propn,0.957389,verb noun,2.328096,noun verb,0.518893,noun propn,1.024158,pron verb intj,0.000000,propn noun verb,1.063619
8,adj verb propn,0.778331,propn propn propn,2.308095,adj intj propn,0.494527,propn num,1.000000,pron verb intj noun,0.000000,propn propn noun,1.014859
9,propn noun noun,0.754532,propn noun noun,1.952604,pron adj intj propn,0.494527,propn propn,0.909595,pron verb intj pron,0.000000,adj adj,0.969591


,DH POS-SEQ,DH value,HS POS-SEQ,HS value,SH POS-SEQ,SH value,TR POS-SEQ,TR value,OT POS-SEQ,OT value,TX POS-SEQ,TX value
0,emoji emoji,2.935739,adj adj,0.000000,intj noun verb,0.420836,adj adj,0.000000,adj adj,0.000000,noun verb,0.950447
1,emoji emoji emoji,2.581518,propn adv verb,0.000000,intj noun verb pron,0.420836,propn adv verb,0.000000,propn adv verb,0.000000,propn noun verb,0.740133
2,noun verb,2.442150,propn emoji emoji propn,0.000000,noun verb pron noun,0.401891,propn emoji emoji propn,0.000000,propn emoji emoji propn,0.000000,emoji emoji,0.566686
3,propn noun verb,2.220400,propn emoji emoji emoji,0.000000,verb pron noun,0.350559,propn emoji emoji emoji,0.000000,propn emoji emoji emoji,0.000000,propn noun,0.456621
4,emoji emoji emoji emoji,2.192241,propn emoji emoji,0.000000,noun verb pron,0.333377,propn emoji emoji,0.000000,propn emoji emoji,0.000000,verb noun,0.418828
5,propn noun,1.872125,propn emoji,0.000000,intj noun,0.276408,propn emoji,0.000000,propn emoji,0.000000,emoji emoji emoji,0.404945
6,propn propn,1.676521,propn adv verb verb,0.000000,pron noun,0.275358,propn adv verb verb,0.000000,propn adv verb verb,0.000000,adj num noun verb,0.398622
7,verb noun,1.343295,propn adv verb pron,0.000000,verb pron,0.254051,propn adv verb pron,0.000000,propn adv verb pron,0.000000,num noun verb verb,0.398622
8,verb verb,1.223613,propn adv verb noun,0.000000,noun verb,0.182975,propn adv verb noun,0.000000,propn adv verb noun,0.000000,adj num noun,0.358068
9,noun verb verb,1.090063,propn adv verb adv,0.000000,propn adv pron adv,0.000000,propn adv verb adv,0.000000,propn adv verb adv,0.000000,adj num,0.337267


,DH POS-SEQ,DH value,HS POS-SEQ,HS value,SH POS-SEQ,SH value,TR POS-SEQ,TR value,OT POS-SEQ,OT value,TX POS-SEQ,TX value
0,noun verb,8.086229,propn verb,34.131710,noun noun,1.525456,verb noun,5.072599,verb propn,2.000000,noun verb,7.405736
1,noun noun,6.592175,propn noun,12.951542,verb noun,1.187385,propn verb noun propn,4.783142,verb noun,0.954699,verb noun,4.385398
2,propn propn,4.978664,noun noun,10.166820,verb verb,1.010385,propn propn verb noun,4.740328,verb noun emoji,0.737692,propn noun,4.353188
3,propn noun,4.645020,propn propn,7.193580,adj propn,1.000000,verb noun propn noun,4.694076,noun emoji,0.590322,noun noun,4.075890
4,adj noun,3.258693,noun verb,3.919480,propn adj,1.000000,verb noun propn,4.402653,verb noun verb,0.580263,noun verb noun,2.633135
5,propn verb,3.205978,propn adj,3.367177,propn noun,1.000000,propn verb noun,4.186781,pron verb verb noun,0.569266,adj noun,2.254713
6,emoji emoji,2.921750,propn propn propn,2.976963,propn verb noun,0.718125,noun propn noun,3.880215,pron verb verb,0.472528,adj noun verb,2.184423
7,verb noun,2.851267,adv verb,2.444856,noun verb,0.710004,propn propn verb,3.833588,verb verb noun,0.416078,verb propn,2.113169
8,emoji emoji emoji,2.621907,verb noun,2.306811,verb verb noun,0.704536,noun propn,3.799209,noun verb,0.412201,noun noun noun,1.833341
9,noun verb noun,2.326607,adj noun,2.146692,noun verb verb,0.637880,propn verb,3.514557,noun verb noun verb,0.385244,propn noun verb,1.808318


,DH POS-SEQ,DH value,HS POS-SEQ,HS value,SH POS-SEQ,SH value,TR POS-SEQ,TR value,OT POS-SEQ,OT value,TX POS-SEQ,TX value
0,propn propn,77.776816,propn propn,109.221479,propn propn,4.026753,propn propn,60.100032,propn propn,9.487562,propn propn,54.484866
1,noun noun,48.253801,noun noun,69.914266,propn verb,2.734812,noun noun,37.173995,noun verb,7.635864,noun noun,38.304246
2,propn noun,43.157949,propn noun,65.146213,verb noun,2.601473,propn noun,35.996719,propn propn propn,6.144516,propn noun,36.307084
3,propn verb,36.581445,propn propn propn,55.408401,propn propn propn,2.366772,propn verb,34.044675,verb noun,5.617724,noun verb,30.948411
4,propn propn propn,32.292415,noun propn,51.475139,noun propn,2.002998,verb noun,33.783823,noun propn,4.503679,propn verb,28.534659
5,noun verb,31.846627,propn verb,47.635786,propn noun,1.968572,noun verb,33.234351,propn noun,4.461223,propn propn propn,25.519527
6,verb noun,30.424357,noun verb,47.185494,noun noun,1.557902,propn propn propn,28.618319,noun verb noun,4.289634,verb noun,23.593514
7,noun propn,25.061074,verb noun,39.492641,propn verb pron,1.388027,noun propn,26.540181,propn verb,4.156754,noun propn,22.584717
8,verb propn,23.100146,verb propn,34.811183,verb pron,1.118999,verb propn,24.781027,verb propn,3.703613,verb propn,20.232338
9,adj noun,22.358148,adj noun,34.012111,propn propn propn propn,1.040514,adj noun,17.614181,noun noun,3.233056,adj noun,18.338233


# 0_FOR_ANALYSIS_RESULTS

In [ ]:
top_dfs = []
for file in files:
    vals = {}
    df = pd.read_csv(src_folder+file, index_col = 0)
    features = df.columns.difference(labels)
    dfs = []
    for label in labels:
        dl = df[df[label] == 1][features]
        dl = dl.T.sort_values(by=[dl.T.columns[0]], ascending = False).head(10)
        dl.index.name = "".join([w[0] for w in label.split()])
        if label == "Trolling":
            dl.index.name += "R"
        elif label == "Toxic":
            dl.index.name += "X"
        elif label == "Others":
            dl.index.name += "T"
        dl = dl.rename({dl.columns[0]:  dl.index.name + " value"}, axis='columns')
        dl.index.name += " words"
        dfs.append(dl)
    dfs = [d.reset_index(drop=False) for d in dfs]
    x = pd.concat(dfs, axis=1)
    display(x.style.set_caption(file))   
    x.to_csv(dest_folder+"TOP_WORDS_"+file)